# Hidden Markov Models

In [66]:
import numpy as np
import pandas as pd

## 1. Forward algorithm - evaluation problem

Naive way: go over every single possible combinations of hidden states

See excel for visualised tree diagram

In [49]:
# Initial state probabilities
initial_prob = np.array([0.6, 0.4])  # TIRED, HAPPY

# State transition matrix
transition_prob = np.array([
    [0.7, 0.3],  # TIRED -> [TIRED, HAPPY]
    [0.4, 0.6]   # HAPPY -> [TIRED, HAPPY]
])
states = [0, 1]  # indices of [TIRED, HAPPY]

# Emission probabilities
emission_prob = np.array([
    [0.3, 0.5, 0.2],  # TIRED -> [FAIL, OK, PERFECT]
    [0.1, 0.5, 0.4]   # HAPPY -> [FAIL, OK, PERFECT]
])

# Observation sequence: OK -> FAIL -> PERFECT
observations = [1, 0, 2]

In [ ]:
# t1 =================
# start with first state being TIRED (out of 2 states: TIRED 0.6 AND HAPPY 0.4)
# prob of getting an OK is 0.5, i.e. emission_prob[0, 1] # 1 stands for ok
# t2 =================
# second state = transition_prob[0] (first row, either TIRED to TIRED or TIRED to HAPPY)
# let's start with the first state in the first row: tired to TIRED
# prob of getting a FAIL is 0.3, i.e. emission_prob[0, 0] # 0 stands for fail
# t3 =================
# third state = transition_prob[0] (first row, either TIRED to TIRED or TIRED to HAPPY)
# let's start with the first state in the first row: tired to TIRED
# prob of getting a PERFECT is 0.2, i.e. emission_prob[0, 2] # 2 stands for perfect

# total prob of scenario ==================
# = 0.5 * 0.3 * 0.2 = 0.03

# back to t3 =================
# let's move on to the second state in the first row: tired to HAPPY
# back to t3 =================
# let's move on to the first state in the second row: happy to TIRED
# back to t3 =================
# let's move on to the second state in the second row: happy to HAPPY

# back to t2 =================
# let's start with the second state in the first row: tired to HAPPY
# back to t2 =================
# let's start with the first state in the second row: happy to TIRED
# back to t2 =================
# let's start with the second state in the second row: happy to HAPPY

# back to t1 =================
# let's start with the second state in the first row: tired to HAPPY
# back to t1 =================
# let's start with the first state in the second row: happy to TIRED
# back to t1 =================
# let's start with the second state in the second row: happy to HAPPY

In [ ]:
for s1 in states: 
    for s2 in states:
        for s3 in states:
            

line 1

In [71]:
initial_prob[0] # 0.6 - tired

emission_prob[0, observations[0]] # 0.5 - ok

0.5

line 2

In [73]:
transition_prob[0, 0]

0.7

In [75]:
emission_prob[0][observations[1]] # tired -> fail: 0.3

0.3

In [77]:
prob_ok_fail_perfect = 0

for s1 in states:
    print(s1)
    for s2 in states:
        print(s2)
        for s3 in states:
            print(s3)
            prob = initial_prob[s1] * emission_prob[s1, observations[0]]
            # 1: tired 0.6 * ok 0.5 = 0.3
            prob *= transition_prob[s1, s2] * emission_prob[s2][observations[1]]
            # *= is the same as prob = prob * ...

0
0
0
1
1
0
1
1
0
0
1
1
0
1


In [47]:
for s1 in states:
    for s2 in states:
        for s3 in states:
            # Probability of the initial state and first observation
            prob = initial_prob[s1] * emission_prob[s1, observations[0]]
            # Transition from s1 to s2 and second observation
            prob *= transition_prob[s1, s2] * emission_prob[s2, observations[1]]
            # Transition from s2 to s3 and third observation
            prob *= transition_prob[s2, s3] * emission_prob[s3, observations[2]]
            # Add the probability of this sequence to the total
            total_prob += prob

print(f"The probability of observing the sequence OK -> FAIL -> PERFECT is: {total_prob}")


The probability of observing the sequence OK -> FAIL -> PERFECT is: 0.029339999999999995


Forward algorithm

In [48]:
# Initial state probabilities
initial_prob = np.array([0.6, 0.4])  # TIRED, HAPPY

# State transition matrix
transition_prob = np.array([
    [0.7, 0.3],  # TIRED -> [TIRED, HAPPY]
    [0.4, 0.6]   # HAPPY -> [TIRED, HAPPY]
])

# Emission probabilities
emission_prob = np.array([
    [0.3, 0.5, 0.2],  # TIRED -> [FAIL, OK, PERFECT]
    [0.1, 0.5, 0.4]   # HAPPY -> [FAIL, OK, PERFECT]
])

# Observation sequence: OK -> FAIL -> PERFECT
observations = [1, 0, 2]

# Forward algorithm
# Initialize alpha values
alpha = np.zeros((len(observations), len(initial_prob)))

# Initial step
for s in range(len(initial_prob)):
    alpha[0, s] = initial_prob[s] * emission_prob[s, observations[0]]

# Recursion step
for t in range(1, len(observations)):
    for s in range(len(initial_prob)):
        alpha[t, s] = np.sum(alpha[t - 1] * transition_prob[:, s]) * emission_prob[s, observations[t]]

# Termination step: sum of the final alpha values
probability = np.sum(alpha[-1])

print(f"The probability of observing the sequence OK -> FAIL -> PERFECT is: {probability}")

The probability of observing the sequence OK -> FAIL -> PERFECT is: 0.02934
